In [1]:
# ! pip install openai
# ! pip install git+https://github.com/openai/whisper.git
# ! pip install pytube
# ! pip install pypdf langchain

## 1. Papers

In [2]:
import os
import sys
sys.path.append("../")

import pandas as pd
from tqdm import tqdm

from langchain.document_loaders import PyPDFLoader

from utils.arxiv_utils import get_inspire_hep_papers, extract_arxiv_ids, download_arxiv_source, remove_latex_preamble
from utils.db_utils import update_dataframe, delete_files_except_extensions, get_filenames_with_extensions, scrape_website_text

%load_ext autoreload
%autoreload 2

In [3]:
pdf_dir = '../data/papers/'  # directory to store PDFs
db_dir = '../data/db/'  # directory to store database CSVs
txt_dir = '../data/interviews/'  # directory to store interview transcripts

In [4]:
try:
    os.makedirs(pdf_dir)
except FileExistsError:
    print(f"Directory '{pdf_dir}' already exists")

# Get papers from INSPIRE-HEP
papers = get_inspire_hep_papers("Jesse.Thaler.1", 2021) 

# Extract arXiv IDs from papers
arxiv_ids = extract_arxiv_ids(papers)

# Download papers (sources if available, otherwise PDFs)
[download_arxiv_source(arxiv_id, output_dir=pdf_dir) for arxiv_id in tqdm(arxiv_ids)];  

Directory '../data/papers/' already exists


  4%|▍         | 1/26 [00:05<02:16,  5.47s/it]

Found 1 .tex files in 2112.05722
They are: ['../data/papers/2112.05722/main.tex']
Using ../data/papers/2112.05722/main.tex as the main .tex file


  8%|▊         | 2/26 [00:22<05:00, 12.50s/it]

Found 1 .tex files in 2205.02857
They are: ['../data/papers/2205.02857/celestial_nongaussianity.tex']
Using ../data/papers/2205.02857/celestial_nongaussianity.tex as the main .tex file


...

Found 1 .tex files in 2008.08596
They are: ['../data/papers/2008.08596/topics_draft.tex']
Using ../data/papers/2008.08596/topics_draft.tex as the main .tex file


 96%|█████████▌| 25/26 [05:08<00:15, 15.76s/it]

Found 1 .tex files in 2010.11998
They are: ['../data/papers/2010.11998/main_v2_arXiv.tex']
Using ../data/papers/2010.11998/main_v2_arXiv.tex as the main .tex file


100%|██████████| 26/26 [05:24<00:00, 12.49s/it]

Found 1 .tex files in 2101.08320
They are: ['../data/papers/2101.08320/merge_rewrite.tex']
Using ../data/papers/2101.08320/merge_rewrite.tex as the main .tex file


In [5]:
# Delete all files except PDFs and TeX files; load relevant files list
delete_files_except_extensions(pdf_dir, ['.pdf', '.tex'])

In [19]:
filenames = get_filenames_with_extensions(pdf_dir, ['.tex', '.pdf'])

In [20]:
source_type = []
text = []

for i, file in enumerate(tqdm(filenames)):
    try:
        if os.path.splitext(file)[-1] == '.pdf':
            loader = PyPDFLoader("{}/{}".format(pdf_dir, file))
            pages = loader.load_and_split()
            text.append(''.join([page.page_content for page in pages]))
            source_type.append("paper")
        elif os.path.splitext(file)[-1] == '.tex':
            with open("{}/{}".format(pdf_dir, file), 'r', encoding='iso-8859-1') as f:
                text.append(remove_latex_preamble(f.read()))
                source_type.append("paper")
    except:
        print("Error with file {}".format(file))

100%|██████████| 26/26 [00:00<00:00, 4898.13it/s]


In [21]:
data = [source_type, text]

# Transpose the data to have the inner lists as rows
transposed_data = list(map(list, zip(*data)))

# Column names for the DataFrame
columns = ['source_type','text']

# Create the DataFrame
df = pd.DataFrame(transposed_data, columns=columns)

In [22]:
# Update dataframe
df = update_dataframe('{}/df_text.csv'.format(db_dir), df)
df.to_csv('{}/df_text.csv'.format(db_dir), index=False)

## 2. YouTube videos

In [23]:
# from tqdm import tqdm
# import whisper
# import pytube
# from pathlib import Path
# import subprocess
# import numpy as np

In [24]:
# videos_dir = "../data/videos/"

# try:
#     os.makedirs(videos_dir)
# except FileExistsError:
#     print(f"Directory '{videos_dir}' already exists")

In [25]:
# # Get whisper model; download weights if necessary
# whisper_model = whisper.load_model("tiny.en").to('cpu')
# options = whisper.DecodingOptions(language="en", without_timestamps=True)

# url = "https://www.youtube.com/watch?v=dqxdPNzBY0I"
# pytube_vid = pytube.YouTube(url)
# video_path_local = Path(videos_dir).resolve() / (pytube_vid.video_id+".mp4")
# pytube_vid.streams.filter(type="audio", mime_type="audio/mp4", abr="48kbps").first().download(output_path=video_path_local.parent, filename=video_path_local.name)
# video_path_local = video_path_local.with_suffix(".wav")
# result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])
# transcription = whisper.transcribe(whisper_model, str(video_path_local))

## 3. Interviews

In [26]:
filenames = get_filenames_with_extensions(txt_dir, ['.txt'])

In [27]:
source_type = []
text = []

for i, file in enumerate(tqdm(filenames)):
    try:
        with open("{}/{}".format(txt_dir, file), 'r') as f:
            text.append(f.read())
            source_type.append("interview")
    except:
        print("Error with file {}".format(file))

100%|██████████| 6/6 [00:00<00:00, 5420.16it/s]


In [28]:
data = [source_type, text]

# Transpose the data to have the inner lists as rows
transposed_data = list(map(list, zip(*data)))

# Column names for the DataFrame
columns = ['source_type','text']

# Create the DataFrame
df = pd.DataFrame(transposed_data, columns=columns)

# Update dataframe
df = update_dataframe('{}/df_text.csv'.format(db_dir), df)
df.to_csv('{}/df_text.csv'.format(db_dir), index=False)


## 4. Website/CV

In [29]:
websites = ["https://jthaler.net/group", "https://jthaler.net/research", "https://jthaler.net/engagement", "https://jthaler.net/faq", "https://jthaler.net/cv", "https://jthaler.net/contact"]
text_website = [scrape_website_text(website) for website in tqdm(websites)]

100%|██████████| 6/6 [00:01<00:00,  4.75it/s]


In [30]:
data = [len(text_website) * ["website"], text_website]

# Transpose the data to have the inner lists as rows
transposed_data = list(map(list, zip(*data)))

# Column names for the DataFrame
columns = ['source_type','text']

# Create the DataFrame
df = pd.DataFrame(transposed_data, columns=columns)

# Update dataframe
df = update_dataframe('{}/df_text.csv'.format(db_dir), df)
df.to_csv('{}/df_text.csv'.format(db_dir), index=False)

In [31]:
len(df)

38